# Notebook to calculate group differences based on paper drakesmith et al

This is the  multi-threshold permutation correction approach and consists:

1) Apply thresholds to networks and compute network metrics for all networks across all thresholds
2) Compute test statistics for each network
3) Permute across groups to get the null distrubition
4) Take the maximum test statistic across all thresholds for each permutation resulting in one summarised null statistic for each permutation. 
5) Identify the critical value for the test-statistic from the top αth percentile of the null test statistics, where α is the desired confidence level (e.g., 5%).
6) Identify clusters where the true test statistic is higher than the critical value and compute the AUC for these clusters.
7) Compute a critical AUC from the mean of the super-critical AUCs for the permuted tests (denoted Acrit).
 

In [29]:
import os
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import functions.plotting_functions as Pfun
import functions.statistical_functions as Sfun
import seaborn as sns
sns.set_style('dark')
from decouple import config

data = config('data')

In [30]:
def pval_plotting(ctvalues:list, tcrit_value:int, measure:str, permutations:int, crit_val:int) -> None:
    
    '''
    Wrapper around seaborn displot.
    Function to plot null distribution with 2 std either side and true critical value.

    Parameters
    -----------------------------------------------------------------
    ctvalues : list, list of crticial values from the null distrubtion.
    tcrit_value: int True crtical value of a test.
    measure: str, name of the measure (used in title of the graph)
    permutations: int, number of permuations (used in title of the graph)


    Returns
    -----------------------------------------------------------------
    hist: histogram of null distribution with upper and lower std with
          true value.
    '''
    
    #Calculates the mean and std deviation of null distribution

    mean_val = Sfun.mean_std(ctvalues)
    
    #plots 
    hist = sns.displot(data=ctvalues, height=10).set(title=f'Histplot for {measure} after {permutations} permuatations')
    hist.refline(x=tcrit_value, color='purple')
    hist.refline(x=crit_val, color='green', linestyle='-')
    hist.refline(x=mean_val['lower_2std'], linestyle='-', color='black')
    hist.refline(x=mean_val['upper_2std'], linestyle='-', color='black')
    hist.refline(x=mean_val['lower_1std'], linestyle='-', color='red')
    hist.refline(x=mean_val['upper_1std'], linestyle='-', color='red')
    plt.show()

In [31]:
def list_of_measures() -> list:
    '''
    Function to get list of graph measures

    Parameters
    ----------
    None

    Returns
    -------
    list: list of graph theory measures
    '''
    return ['average_clustering', 'average_shortest_path_length', 'assortativity', 'modularity', 'efficiency']

In [32]:
def permuations(permutations:int, length_of_group_1:int, length_of_group_2:int, participants:pd.DataFrame, names:list, centroids:np.float64, threshold:int) -> dict:
    measures = list_of_measures()
    null_distribution ={
        
        'average_clustering':[],
        'average_shortest_path_length':[], 
        'assortativity':[], 
        'modularity':[], 
        'efficiency':[]
    
    }
    
    for perm in range(permutations):
        group_1_participants = participants.sample(n=length_of_group_1)
        group_2_participants = participants.sample(n=length_of_group_2)
        
        group_1_graphs = Sfun.create_graphs(group_1_participants.iloc[:,1:69], names, centroids, threshold=threshold)
        group_2_graphs = Sfun.create_graphs(group_2_participants.iloc[:,1:69], names, centroids)
        
        group_1_values = group_1_graphs['graph_threshold'].calculate_global_measures()
        group_2_graphs = group_2_graphs['graph_threshold'].calculate_global_measures()
    
        for meas in measures:
            crit_val = group_1_values[meas] -  group_2_graphs[meas]
            null_distribution[meas].append(crit_val)
            
    return null_distribution

In [33]:
def find_max_null_stat(threshold_value:list, null_distribution:dict, group_key:str, measure_key:str, list_number:int) -> dict:
    
    '''
    Function to loop through each element in a list for each thresold value.

    Parameters
    ----------
    null_distribution:dict dictionary of null_distribution
    group_key:str dictionary key for group 
    measure_key:str dictionary key for the graph theory measure.
    list_number:int list index of permutation

    Returns
    -------
    max_null_statistic:int max null statistic for that permutation.

    '''
    
    values = []
    for threshold in threshold_value:
        values.append(null_distribution[group_key][f'thresholded_value_{threshold}'][measure_key][list_number])
        
    max_null_statistic = max(values, key=abs)

    max_null_dict = {
        'summarised_values':values,
        'max_null':max_null_statistic
    }
    
    return max_null_dict

Import data and establish measures.

In [34]:
lh_volume = pd.read_csv(f'{data}/lh_volume.dat',sep='\t').drop(['BrainSegVolNotVent', 'eTIV'],axis=1).rename(columns={'lh.aparc.volume':'G-Number'})
rh_volume =  pd.read_csv(f'{data}/rh_volume.dat',sep='\t').drop(['BrainSegVolNotVent', 'eTIV','rh.aparc.volume'],axis=1)

group = pd.read_csv(f'{data}/cortical_measures.csv').iloc[0:,2]

volume = pd.concat([lh_volume, rh_volume, group],axis=1)

names = list(volume.columns.drop(['G-Number','age_adjusted_group']))

centroids = pd.read_csv(f'{data}/atlas.csv')
centroids = centroids[['x.mni', 'y.mni', 'z.mni']].to_numpy()


group = volume.groupby('age_adjusted_group')
aan = group.get_group('AAN').reset_index(drop=True)
hc = group.get_group('HC').reset_index(drop=True)
wr = group.get_group('WR').reset_index(drop=True)

measures = list_of_measures()

## 1) Apply thresholds to networks and compute network metrics for all networks across all thresholds

First the threshold range needs to be decided. To do this the methodology used is taken from Bassett et al (https://doi.org/10.1073/pnas.0606005103) where the upper limit must not exceed 2 * natural log(nodes). 
Starting point of 4 is used otherwise minimum spanning tree is too large and throws up error message.

Threshold values are stored in a list.

In [35]:
threshold_value=[]
for threshold in range(4 , 100):
    aan_graphs = Sfun.create_graphs(aan.iloc[:,1:69], names, centroids, threshold=threshold)
    aan_graphs['graph_threshold'].calculate_nodal_measures()
    cal = aan_graphs['graph_threshold'].report_nodal_measures()
    
    if cal['degree'].mean() > 2 * np.log(len(aan_graphs['graph_threshold'].nodes())):
        break
    
    threshold_value.append(threshold)

threshold_value = range(4,100)

Now create graphs at the pre-defined thresholds.

In [36]:
result = {
    'aan_graphs':[],
    'hc_graphs':[],
    'wr_graphs':[]
}

In [37]:
for threshold in threshold_value:
    aan_graphs = Sfun.create_graphs(aan.iloc[:,1:69], names, centroids, threshold=threshold)
    aan_graphs['graph_threshold'].__name__ = f'aan_graph_threshold_value_{threshold}'
    result['aan_graphs'].append(aan_graphs['graph_threshold'])

    hc_graphs = Sfun.create_graphs(hc.iloc[:,1:69], names, centroids, threshold=threshold)
    hc_graphs['graph_threshold'].__name__ = f'hc_graph_threshold_value_{threshold}'
    result['hc_graphs'].append(hc_graphs['graph_threshold'])

    wr_graphs = Sfun.create_graphs(wr.iloc[:,1:69], names, centroids, threshold=threshold)
    wr_graphs['graph_threshold'].__name__ = f'wr_graph_threshold_value_{threshold}'
    result['wr_graphs'].append(wr_graphs['graph_threshold'])

In [38]:
len(result['aan_graphs'])

9

Finally calculate global measures for each graph at the threshold ranges.

In [39]:
global_measures_dict = {}

In [40]:
for keys, values in result.items():
    for graph_object in values:
        global_measures = graph_object.calculate_global_measures()
        global_measures_dict[f'{graph_object.__name__}'] = []
        global_measures_dict[f'{graph_object.__name__}'].append(global_measures)

## 2) Compute test statistics for each network

Calculate the test statistic for each condition. This is going to be the difference between each global measure for two groups. 

In [41]:
test_statistics = {
    'aan_hc':{},
    'wr_hc':{},
    'wr_aan':{}
}

In [42]:
for threshold in threshold_value:
    for keys, value in global_measures_dict['aan_graph_threshold_value_4'][0].items():
        statistic_aan_hc =  global_measures_dict[f'aan_graph_threshold_value_{threshold}'][0][keys] - global_measures_dict[f'hc_graph_threshold_value_{threshold}'][0][keys]
        statistic_wr_hc =  global_measures_dict[f'wr_graph_threshold_value_{threshold}'][0][keys] - global_measures_dict[f'hc_graph_threshold_value_{threshold}'][0][keys]
        statistic_wr_aan =  global_measures_dict[f'wr_graph_threshold_value_{threshold}'][0][keys] - global_measures_dict[f'aan_graph_threshold_value_{threshold}'][0][keys]
        
        key = f'{keys}_at_threshold_value_{threshold}'
      
        test_statistics['aan_hc'][key] = statistic_aan_hc 
        test_statistics['wr_hc'][key] = statistic_wr_hc
        test_statistics['wr_aan'][key] = statistic_wr_aan

In [43]:
for group_key in test_statistics.keys():
    for measure in measures:
        measure_summary = [test_statistics[group_key][key] for key in test_statistics[group_key] if measure in key]
        test_statistics[group_key][measure] = measure_summary

## 3) Permute across groups to get the null distrubition

Now we permutate to create a null distrubtion


aan_hc_df = pd.concat([aan, hc], ignore_index=True)
aan_wr_df = pd.concat([wr, aan], ignore_index=True)
wr_hc_df = pd.concat([wr, hc], ignore_index=True)

null_distribution = {
    'aan_hc':{},
    'wr_hc':{},
    'wr_aan':{}
}

for threshold in threshold_value:
    print('thresholding null at:', threshold )
    aan_hc_perm = permuations(10, len(aan['G-Number']), len(hc['G-Number']), aan_hc_df, names, centroids, threshold)
    aan_wr_perm = permuations(10, len(aan['G-Number']), len(wr['G-Number']), aan_wr_df, names, centroids, threshold)
    wr_hc_perm = permuations(10, len(wr['G-Number']), len(hc['G-Number']), wr_hc_df, names, centroids, threshold)
    
    key = f'thresholded_value_{threshold}'
    null_distribution['aan_hc'][key] = aan_hc_perm 
    null_distribution['wr_hc'][key] = aan_wr_perm
    null_distribution['wr_aan'][key] = wr_hc_perm

import pickle

with open('permutations_vol_all_thresholds.pickle', 'wb') as handle:
    pickle.dump(null_distribution, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [44]:
import pickle

with open('permutations.pickle', 'rb') as handle:
    null_distribution = pickle.load(handle)

In [45]:
print(max(null_distribution['aan_hc']['thresholded_value_4']['average_shortest_path_length']))
print(test_statistics['aan_hc']['average_shortest_path_length_at_threshold_value_4'])

4.231782265144864
-0.08077260755048243


## 4) Take the maximum test statistic across all thresholds for each permutation resulting in one summarised null statistic for each permutation. 



In [46]:
null_stat_summarized = {
    
    'aan_hc':{},
    'wr_hc':{},
    'wr_aan':{}
}


Once dictionary has been initialised it needs to be set up to be able to take the permutations and list.

In [47]:
for group_key in null_stat_summarized.keys():
    for measure in measures:
        null_stat_summarized[group_key][measure] = []

In [48]:
for group_key in null_stat_summarized.keys():
    for perm in range(0, 100):
        for measure in measures:
            max_null_statistic = find_max_null_stat(threshold_value, null_distribution, group_key, measure, perm )
            null_stat_summarized[group_key][measure].append(max_null_statistic['max_null'])

## 5) Identify the critical value for the test-statistic from the top αth percentile of the null test statistics, where α is the desired confidence level (e.g., 5%).

To take into account direction of effect the Max value is calculated at both ends of the distribution.   

In [49]:
crit_val = {
        
    'aan_hc':{},
    'wr_hc':{},
    'wr_aan':{}
}

In [50]:
for group_key in test_statistics.keys():
    for measure in measures:
        if max(test_statistics[group_key][measure]) > 0:
            if min(test_statistics[group_key][measure]) < 0:
                crit_val[group_key][measure] = np.abs(np.quantile(null_stat_summarized[group_key][measure], 0.975))

            if min(test_statistics[group_key][measure]) > 0:
                crit_val[group_key][measure] = np.quantile(null_stat_summarized[group_key][measure], 0.95)

        if max(test_statistics[group_key][measure]) <= 0:
            crit_val[group_key][measure] = np.quantile(null_stat_summarized[group_key][measure], 0.05)



## 6) Identify clusters where the true test statistic is higher than the critical value and compute the AUC for these clusters.

This is done by checking that the statistic is greater than the max value on both ends of the distribution.

First work out clusters greater than the max value

In [51]:
clusters = {}

for group_key in test_statistics.keys():
       for key , val in test_statistics[group_key].items():
            new_key = re.sub(r'_at_threshold_value_.*', '', key)
            critical_value = crit_val[group_key][new_key]
            
            if type(val) != list:
                if val > critical_value:
                    clusters[f'{group_key}_{key}'] = {
                                    'value':test_statistics[group_key][key],
                                    'critical_value':critical_value,
                                    'null':null_stat_summarized[group_key][new_key]
                                    
                                    }

In [52]:
print(clusters.keys())

dict_keys(['wr_aan_average_clustering_at_threshold_value_4', 'wr_aan_average_clustering_at_threshold_value_5', 'wr_aan_average_clustering_at_threshold_value_6', 'wr_aan_average_clustering_at_threshold_value_7', 'wr_aan_average_clustering_at_threshold_value_8', 'wr_aan_average_clustering_at_threshold_value_9', 'wr_aan_average_clustering_at_threshold_value_10', 'wr_aan_average_clustering_at_threshold_value_11', 'wr_aan_average_clustering_at_threshold_value_12'])


calculate pvals

In [53]:
pvals = {}

for measure_key in clusters.keys():
    if clusters[measure_key]['value'] > 0 and clusters[measure_key]['critical_value'] > 0:
        sum_list = [val for val in clusters[measure_key]['null'] if val >= clusters[measure_key]['value']]
        pvals[measure_key] = (len(sum_list)/100) 

        
    elif clusters[measure_key]['value'] < 0 and clusters[measure_key]['critical_value'] < 0:
        sum_list = [val for val in clusters[measure_key]['null'] if val <= clusters[measure_key]['value']]
        pvals[measure_key] = (len(sum_list)/100)

    else: 
        sum_list = [val for val in clusters[measure_key]['null'] if val <= np.abs(clusters[measure_key]['value'])]
        pvals[measure_key] = (len(sum_list)/100)


This creates a dictionary with all the threshold values in a list 

In [54]:
check_dic = {}

for group_key in test_statistics.keys():
    for measure in measures:
        check_dic[f'{group_key}_{measure}'] = []
        
        for key, val in clusters.items():
            if measure in key:
                if group_key in key:
                    check_dic[f'{group_key}_{measure}'].append(clusters[key]['value'])

This is taken from leons code. I think it is creating a high spec grid (matrix) then linear interploating it. 

In [55]:
auc = {
    'aan_hc':{},
    'wr_hc':{},
    'wr_aan':{}

}
for group_key in auc.keys():
    for measure in measures:
        if len(check_dic[f'{group_key}_{measure}']) == len(threshold_value):
            threshold_values_linspace = np.linspace(start=min(threshold_value), stop=max(threshold_value), num=500)
            observation_interploated = np.interp(threshold_values_linspace, threshold_value, check_dic[f'{group_key}_{measure}'])
            obs_AUC = np.trapz(threshold_values_linspace, observation_interploated) 
            auc[group_key][measure] = obs_AUC


## 7) Compute a critical AUC from the mean of the super-critical AUCs for the permuted tests (denoted Acrit).

This does exactly the same as for the 

In [56]:
summarized_permutations = {
    
    'aan_hc':{},
    'wr_hc':{},
    'wr_aan':{}
}

for group_key in summarized_permutations.keys():
    for measure in measures:
        summarized_permutations[group_key][measure] = []

for group_key in summarized_permutations.keys():
    for perm in range(0, 100):
        for measure in measures:
            max_null_statistic_values = find_max_null_stat(threshold_value, null_distribution, group_key, measure, perm )
            summarized_permutations[group_key][measure].append(max_null_statistic_values['summarised_values'])

In [62]:
print('hi')
print('hello')
print('hill')

hi
hello
hill


In [57]:
null_auc ={
    'aan_hc':{},
    'wr_hc':{},
    'wr_aan':{}
}

for group_key in auc.keys():
    for measure in measures:
        null_auc[group_key][measure] = []
        for perm in range(0, 100):
            if len(check_dic[f'{group_key}_{measure}']) == len(threshold_value):
                threshold_values_linspace = np.linspace(start=min(threshold_value), stop=max(threshold_value), num=500)
                observation_interploated = np.interp(threshold_values_linspace, threshold_value, summarized_permutations[group_key][measure][perm])
                obs_AUC = np.trapz(threshold_values_linspace, observation_interploated) 
                null_auc[group_key][measure].append(obs_AUC)